In [3]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gc
import time
import pytz
import datetime
import logging
from tqdm import tqdm
from tqdm.contrib import tqdm
import traceback

In [4]:
df = pd.read_excel('Indeed_Extract_20231023.xlsx')

In [5]:
skilldf = pd.read_excel('role_wise_top_skills .xlsx')

In [6]:
skilldf = skilldf.iloc[:,::2]

In [7]:
del skilldf['Business Analyst']
del skilldf['Recruiter']
del skilldf['Lead/Management']	
del skilldf['Counsellor']	

In [8]:
skills = set()
for i in range(len(skilldf.columns)):
    l = skilldf.iloc[:,i].to_list()
    skills.update(set(l))

In [23]:
def clean_string(s):
    if isinstance(s, str):
        s = re.sub(r'[^A-Za-z0-9]', ' ', s)
        s = re.sub(r'\s+', ' ', s)
        return s.lower()
    else:
        return s 

df['description'] = df['description'].apply(clean_string)

In [24]:
df['description'][1]

'379766br qa operations associate usa about the role quality assurance qa operations associate location millburn nj li onsite about this role at advanced accelerator applications a novartis company we are committed to leading innovation in nuclear medicine and delivering the next generation of targeted radioligand therapy to cancer patients we are looking for an experienced quality assurance operations associate to help us reach our ambitious goals the quality assurance associate will be responsible for oversight of manufacturing testing and supply chain operations to ensure compliance with current gmp regulations procedures and systems key responsibilities work closely with the production quality control and supply chain departments to ensure their practice fully adheres to cgmp responsible for authoring revising and approving standard operating procedures policies and records responsible for support of manufacturing operations such as batch record issuance and reconciliation activiti

In [25]:
skills_pattern = re.compile(r'\b(?:' + '|'.join(re.escape(skill) for skill in skills) + r')\b', flags=re.IGNORECASE)

def extract_skills_parallel(description):
    if isinstance(description, str):
        skills_found = list(set(re.findall(r'\b\w+\b', description, flags=re.IGNORECASE)) & set(skills))
        return ', '.join(skills_found)  # Return a comma-separated string of skills
    else:
        return '' 

total_rows = len(df)
progress_bar = tqdm(total=total_rows, desc="Processing")

# Apply the extract_skills_parallel function and update progress bar
for index, row in df.iterrows():
    df.at[index, 'skills'] = extract_skills_parallel(row['description'])
    progress_bar.update(1)

Processing:   0%|          | 0/3904 [00:00<?, ?it/s]

Processing:  96%|█████████▌| 3753/3904 [00:02<00:00, 1905.58it/s]

In [ ]:
df.to_excel('indeed_skills_extraction.xlsx',index=False)

In [59]:
df.columns

Index(['job_title', 'company_name', 'salary', 'location', 'description',
       'posted_on', 'current_date', 'skill1', 'skill2', 'skill3', 'skill4',
       'skill5', 'skill6', 'skill7', 'skill8', 'experience', 'skill9',
       'skills'],
      dtype='object')

In [62]:
df = df.reindex(columns = ['job_title', 'company_name', 'salary', 'location', 'description',
       'posted_on', 'current_date', 'skill1', 'skill2', 'skill3', 'skill4',
       'skill5', 'skill6', 'skill7', 'skill8', 'skill9', 'skills','experience'])

In [64]:
df.to_excel('indeed_skills_extraction.xlsx',index=False)

In [25]:
df.to_csv('comp.csv',index=False)

In [30]:
df

,job_title,company_name,salary,location,description,posted_on,current_date,skill1,skill2,skill3,skill4,skill5,skill6,skill7,skill8,experience,skill9,skills
0,"Associate Partner, Human Resources",Amazon.com Services LLC,"Up to $71,200 a year","Tracy, CA",NaN,Today,2023-10-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,entry level,NaN,NaN
1,QA Operations Associate,Novartis,"$59,900 - $89,900 a year","Millburn, NJ 07041",379766BR\nQA Operations Associate\nUSA\nAbout ...,Posted 30+ days ago,2023-10-18,Project management,CGMP,Bachelor's degree,NaN,NaN,NaN,NaN,NaN,entry level,NaN,<s>
2,Secretary (OA),US Internal Revenue Service,"$57,118 - $91,873 a year","Hybrid remote in Phoenix, AZ",Duties\nWHAT IS THE LBI - LARGE BUSINESS AND I...,Posted 5 days ago,2023-10-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,entry level,NaN,
3,Quality Assurance Specialist,US Assistant Secretary for Public and Indian...,"$98,496 - $158,432 a year","Wallops Island, VA","Duties\nAs a Quality Assurance Specialist, you...",Posted 6 days ago,2023-10-18,Section 8,Negotiation,NaN,NaN,NaN,NaN,NaN,NaN,entry level,NaN,<s>
4,Gift Planning Associate,The Nature Conservancy,"$68,935 - $76,234 a year","Hybrid remote in Arlington, VA","OFFICE LOCATION\n\nArlington, Virginia, USA\n\...",Posted 25 days ago,2023-10-18,Research,Recruiting,Proofreading,Databases,Bachelor's degree,Multilingual,NaN,NaN,entry level,NaN,<s>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2888,Packaging Engineer,Broadcom,"$126,000 - $210,000 a year","San Jose, CA","Please Note:\n1. If you are a first time user,...",Posted 30+ days ago,2023-10-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,senior level,NaN,NaN
2889,Clinical Affairs Senior Manager,Grifols,"$145,000 - $181,000 a year","San Diego, CA 92102",Do you want to join an international team work...,Posted 30+ days ago,2023-10-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,senior level,NaN,NaN
2890,"Executive Director, Scientific Affairs â€“ Car...",Merck Sharp & Dohme,"$259,440 - $408,300 a year","Hybrid remote in Rahway, NJ 07065",Job Description\nOur Medical Affairs team adva...,Posted 23 days ago,2023-10-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,senior level,NaN,NaN
2891,Sr. Process Engineer,Career Technologies USA,"$130,000 - $160,000 a year","Chatsworth, CA","Full Time + Benefits (Medical, Dental, Vision,...",Posted 30+ days ago,2023-10-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,senior level,NaN,NaN


In [25]:
df['description']

0                                                     NaN
1       379766BR\nQA Operations Associate\nUSA\nAbout ...
2       Duties\nWHAT IS THE LBI - LARGE BUSINESS AND I...
3       Duties\nAs a Quality Assurance Specialist, you...
4       OFFICE LOCATION\n\nArlington, Virginia, USA\n\...
                              ...                        
2888    Please Note:\n1. If you are a first time user,...
2889    Do you want to join an international team work...
2890    Job Description\nOur Medical Affairs team adva...
2891    Full Time + Benefits (Medical, Dental, Vision,...
2892    Job Description\nOur Development team ensure p...
Name: description, Length: 2893, dtype: object

"379766BR\nQA Operations Associate\nUSA\nAbout the role\nQuality Assurance(QA) Operations Associate\n\nLocation: Millburn, NJ #LI-Onsite\n\nAbout this role:\n\nAt Advanced Accelerator Applications, a Novartis company, we are committed to leading innovation in nuclear medicine and delivering the next generation of targeted radioligand therapy to cancer patients. We are looking for an experienced Quality Assurance Operations Associate to help us reach our ambitious goals.\n\nThe Quality Assurance Associate will be responsible for oversight of manufacturing, testing and supply chain operations to ensure compliance with current GMP regulations, procedures and systems.\n\nKey Responsibilities:\n\nWork closely with the production, quality control and supply chain departments to ensure their practice fully adheres to cGMP\nResponsible for authoring, revising and approving standard operating procedures, policies and records\nResponsible for support of manufacturing operations such as batch rec